### Loading Packages and Data

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 8)

#import xarray as xr
import warnings
warnings.filterwarnings('ignore')
from matplotlib.backends.backend_pdf import PdfPages

from db_queries import get_outputs as go
from db_queries import get_ids
from get_draws.api import get_draws

import altair as alt
from vega_datasets import data

In [2]:
ls /mnt/team/simulation_science/pub/models/vivarium_nih_us_cvd/results/final-runs-test/nih_us_cvd/2023_12_18_15_45_33/count_data

binned_ldl_exposure_time.csv     risk_exposure_time.csv
binned_sbp_exposure_time.csv     sbp_medication_person_time.csv
deaths.csv                       state_person_time.csv
healthcare_visits.csv            transition_count.csv
intervention_person_time.csv     ylds.csv
ldlc_medication_person_time.csv  ylls.csv


In [3]:
sim_results_dir = '/mnt/team/simulation_science/pub/models/vivarium_nih_us_cvd/results/final-runs-test/nih_us_cvd/2023_12_18_15_45_33/count_data'

## Checking Stability of Risks over Time

In [4]:
state_person_time = pd.read_csv(sim_results_dir + '/state_person_time.csv')
person_time = state_person_time.loc[state_person_time['state'].isin(["acute_ischemic_stroke", "chronic_ischemic_stroke", "susceptible_to_ischemic_stroke"])].reset_index() 
person_time = person_time.rename(columns = {'value':'ptvalue'})
per_time_group = person_time.groupby(['age', 'sex', 'year','input_draw','scenario','location']).ptvalue.sum().reset_index()
per_time_group.head()

,age,sex,year,input_draw,scenario,location,ptvalue
0,25_to_29,Female,2021,829,baseline,alabama,3832.914442
1,25_to_29,Female,2021,829,baseline,alaska,3816.739220
2,25_to_29,Female,2021,829,baseline,arizona,3893.552361
3,25_to_29,Female,2021,829,baseline,arkansas,3723.214237
4,25_to_29,Female,2021,829,baseline,california,4161.861739


# SBP Medications

In [5]:
meds = pd.read_csv(sim_results_dir + '/sbp_medication_person_time.csv')
meds.head()

,Unnamed: 0,sex,year,measure,input_draw,scenario,location,age,medication_adherence,sbp_medication,value
0,0,Female,2021,sbp_medication_person_time,829,baseline,alabama,25_to_29,primary_non_adherent,no_treatment,560.996578
1,1,Female,2021,sbp_medication_person_time,829,baseline,alaska,25_to_29,primary_non_adherent,no_treatment,625.390828
2,2,Female,2021,sbp_medication_person_time,829,baseline,arizona,25_to_29,primary_non_adherent,no_treatment,554.097194
3,3,Female,2021,sbp_medication_person_time,829,baseline,arkansas,25_to_29,primary_non_adherent,no_treatment,481.883641
4,4,Female,2021,sbp_medication_person_time,829,baseline,california,25_to_29,primary_non_adherent,no_treatment,649.002053


In [6]:
meds_adherent = meds.groupby(['sex', 'year', 'input_draw', 'age', 'sbp_medication','scenario', 'location']).value.sum().reset_index()
meds_adherent_rate = meds_adherent.merge(per_time_group[['ptvalue','sex','age','year','input_draw','scenario','location']], on=['sex','age','year','input_draw','scenario','location'])
meds_adherent_rate['meds_adherent_rate'] = meds_adherent_rate['value'] / meds_adherent_rate['ptvalue']
meds_adherent_rate_summ = (meds_adherent_rate
                .groupby(['sex', 'year', 'sbp_medication','age','location'])
                .meds_adherent_rate.describe(percentiles=[.025, .975])
                .filter(['mean', '2.5%', '97.5%'])
                .reset_index())
meds_adherent_rate_summ

,sex,year,sbp_medication,age,location,mean,2.5%,97.5%
0,Female,2021,no_treatment,25_to_29,alabama,0.931759,0.931759,0.931759
1,Female,2021,no_treatment,25_to_29,alaska,0.971700,0.971700,0.971700
2,Female,2021,no_treatment,25_to_29,arizona,0.930597,0.930597,0.930597
3,Female,2021,no_treatment,25_to_29,arkansas,0.856366,0.856366,0.856366
...,...,...,...,...,...,...,...,...
214196,Male,2040,two_drug_std_dose_efficacy,95_plus,washington,0.006294,0.000617,0.008475
214197,Male,2040,two_drug_std_dose_efficacy,95_plus,west_virginia,0.017468,0.001739,0.023358
214198,Male,2040,two_drug_std_dose_efficacy,95_plus,wisconsin,0.045757,0.003911,0.065440
214199,Male,2040,two_drug_std_dose_efficacy,95_plus,wyoming,0.023341,0.002316,0.031324


In [7]:
def convert(word):
    return ' '.join(x.capitalize() or '_' for x in word.split('_'))

In [8]:
maps_data_sbp = meds_adherent_rate_summ.loc[(meds_adherent_rate_summ.year == 2030) & (meds_adherent_rate_summ.sbp_medication == 'no_treatment')].reset_index()
maps_data_sbp['total_trt_rate'] = 1 - maps_data_sbp['mean']
maps_data_sbp = maps_data_sbp[['sex','age','location','total_trt_rate']]
maps_data_sbp['location'] = maps_data_sbp.location.apply(convert)
pop = data.population_engineers_hurricanes()
pop= pop.rename(columns = {'state':'location'})
maps_data_sbp = maps_data_sbp.merge(pop[['location','id']], on=['location'])
maps_data_sbp.head()

,sex,age,location,total_trt_rate,id
0,Female,25_to_29,Alabama,0.041088,1
1,Female,30_to_34,Alabama,0.070963,1
2,Female,35_to_39,Alabama,0.167988,1
3,Female,40_to_44,Alabama,0.249474,1
4,Female,45_to_49,Alabama,0.310125,1


In [9]:
def sbp_ages(data_1): 
    charts = []
    for age in data_1.age.unique(): 

        subdata = data_1.loc[(data_1.age == age) & (data_1.sex == 'Female')] 
        states = alt.topo_feature(data.us_10m.url, 'states')
        variable_list = ['total_trt_rate']

        background_1 = alt.Chart(states).mark_geoshape(
            fill='lightgray',
            stroke='white'
        ).project('albersUsa').properties(
            width=500,
            height=300
        )

        overlay_1 = alt.Chart(states).mark_geoshape().encode(
            color=alt.Color('total_trt_rate:Q', title= 'Treatment Rate'),
        ).transform_lookup(
            lookup='id',
            from_=alt.LookupData(subdata, 'id', list(subdata.columns))
        ).properties(
            width=500,
            height=300,
            title=f'SBP Treatment Rate in 2030 for Females, {age}'
        ).project(
            type='albersUsa'
        )

        chart_1 = alt.layer(background_1, overlay_1)
        
        subdata = data_1.loc[(data_1.age == age) & (data_1.sex == 'Male')] 
        states = alt.topo_feature(data.us_10m.url, 'states')
        variable_list = ['total_trt_rate']

        background_2 = alt.Chart(states).mark_geoshape(
            fill='lightgray',
            stroke='white'
        ).project('albersUsa').properties(
            width=500,
            height=300
        )

        overlay_2 = alt.Chart(states).mark_geoshape().encode(
            color=alt.Color('total_trt_rate:Q', title= 'Treatment Rate'),
        ).transform_lookup(
            lookup='id',
            from_=alt.LookupData(subdata, 'id', list(subdata.columns))
        ).properties(
            width=500,
            height=300,
            title=f'SBP Treatment Rate in 2030 for Males, {age}'
        ).project(
            type='albersUsa'
        )

        chart_2 = alt.layer(background_2, overlay_2)

        chart = alt.hconcat(chart_1, chart_2)

        charts.append(chart)
    return charts

In [10]:
sbp_charts = sbp_ages(maps_data_sbp)
alt.vconcat(*sbp_charts).resolve_scale(color="independent")

alt.VConcatChart(...)

# LDL-C Meds

In [11]:
ldl_meds = pd.read_csv(sim_results_dir + '/ldlc_medication_person_time.csv')
ldl_meds.head()

,Unnamed: 0,sex,year,measure,input_draw,scenario,location,age,medication_adherence,ldlc_medication,value
0,0,Female,2021,ldlc_medication_person_time,829,baseline,alabama,25_to_29,primary_non_adherent,high_intensity,0.0
1,1,Female,2021,ldlc_medication_person_time,829,baseline,alaska,25_to_29,primary_non_adherent,high_intensity,0.0
2,2,Female,2021,ldlc_medication_person_time,829,baseline,arizona,25_to_29,primary_non_adherent,high_intensity,0.0
3,3,Female,2021,ldlc_medication_person_time,829,baseline,arkansas,25_to_29,primary_non_adherent,high_intensity,0.0
4,4,Female,2021,ldlc_medication_person_time,829,baseline,california,25_to_29,primary_non_adherent,high_intensity,0.0


In [12]:
meds_adherent_ldl = ldl_meds.groupby(['sex', 'year', 'input_draw', 'age', 'ldlc_medication','scenario', 'location']).value.sum().reset_index()
meds_adherent_rate_ldl = meds_adherent_ldl.merge(per_time_group[['ptvalue','sex','age','year','input_draw','scenario','location']], on=['sex','age','year','input_draw','scenario','location'])
meds_adherent_rate_ldl['meds_adherent_rate'] = meds_adherent_rate_ldl['value'] / meds_adherent_rate_ldl['ptvalue']
meds_adherent_rate_summ_ldl = (meds_adherent_rate_ldl
                .groupby(['sex', 'year', 'ldlc_medication','age','location'])
                .meds_adherent_rate.describe(percentiles=[.025, .975])
                .filter(['mean', '2.5%', '97.5%'])
                .reset_index())
meds_adherent_rate_summ_ldl

,sex,year,ldlc_medication,age,location,mean,2.5%,97.5%
0,Female,2021,high_intensity,25_to_29,alabama,0.004280,0.004280,0.004280
1,Female,2021,high_intensity,25_to_29,alaska,0.002370,0.002370,0.002370
2,Female,2021,high_intensity,25_to_29,arizona,0.003347,0.003347,0.003347
3,Female,2021,high_intensity,25_to_29,arkansas,0.004818,0.004818,0.004818
...,...,...,...,...,...,...,...,...
183596,Male,2040,no_treatment,95_plus,washington,0.497419,0.480265,0.508475
183597,Male,2040,no_treatment,95_plus,west_virginia,0.499717,0.489642,0.504841
183598,Male,2040,no_treatment,95_plus,wisconsin,0.514711,0.511247,0.524061
183599,Male,2040,no_treatment,95_plus,wyoming,0.486517,0.480601,0.490229


In [13]:
maps_data_ldl = meds_adherent_rate_summ_ldl.loc[(meds_adherent_rate_summ_ldl.year == 2030) & (meds_adherent_rate_summ_ldl.ldlc_medication == 'no_treatment')].reset_index()
maps_data_ldl['total_trt_rate'] = 1 - maps_data_ldl['mean']
maps_data_ldl = maps_data_ldl[['sex','age','location','total_trt_rate']]
maps_data_ldl['location'] = maps_data_ldl.location.apply(convert)
pop = data.population_engineers_hurricanes()
pop= pop.rename(columns = {'state':'location'})
maps_data_ldl = maps_data_ldl.merge(pop[['location','id']], on=['location'])
maps_data_ldl.head()

,sex,age,location,total_trt_rate,id
0,Female,25_to_29,Alabama,0.015583,1
1,Female,30_to_34,Alabama,0.020950,1
2,Female,35_to_39,Alabama,0.025107,1
3,Female,40_to_44,Alabama,0.075810,1
4,Female,45_to_49,Alabama,0.112481,1


In [14]:
def ldlc_ages(data_1): 
    charts = []
    for age in data_1.age.unique(): 

        subdata = data_1.loc[(data_1.age == age) & (data_1.sex == 'Female')] 
        states = alt.topo_feature(data.us_10m.url, 'states')
        variable_list = ['total_trt_rate']

        background_1 = alt.Chart(states).mark_geoshape(
            fill='lightgray',
            stroke='white'
        ).project('albersUsa').properties(
            width=500,
            height=300
        )

        overlay_1 = alt.Chart(states).mark_geoshape().encode(
            color=alt.Color('total_trt_rate:Q', title= 'Treatment Rate'),
        ).transform_lookup(
            lookup='id',
            from_=alt.LookupData(subdata, 'id', list(subdata.columns))
        ).properties(
            width=500,
            height=300,
            title=f'LDL-C Treatment Rate in 2030 for Females, {age}'
        ).project(
            type='albersUsa'
        )

        chart_1 = alt.layer(background_1, overlay_1)
        
        subdata = data_1.loc[(data_1.age == age) & (data_1.sex == 'Male')] 
        states = alt.topo_feature(data.us_10m.url, 'states')
        variable_list = ['total_trt_rate']

        background_2 = alt.Chart(states).mark_geoshape(
            fill='lightgray',
            stroke='white'
        ).project('albersUsa').properties(
            width=500,
            height=300
        )

        overlay_2 = alt.Chart(states).mark_geoshape().encode(
            color=alt.Color('total_trt_rate:Q', title= 'Treatment Rate'),
        ).transform_lookup(
            lookup='id',
            from_=alt.LookupData(subdata, 'id', list(subdata.columns))
        ).properties(
            width=500,
            height=300,
            title=f'LDL-C Treatment Rate in 2030 for Males, {age}'
        ).project(
            type='albersUsa'
        )

        chart_2 = alt.layer(background_2, overlay_2)

        chart = alt.hconcat(chart_1, chart_2)

        charts.append(chart)
    return charts

In [15]:
ldl_charts = ldlc_ages(maps_data_ldl)
alt.vconcat(*ldl_charts).resolve_scale(color="independent")

alt.VConcatChart(...)